In [1]:
import os
import pathlib
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
!ls ../input/

age_gender_bkts.csv	   save		   train_users_2.csv
countries.csv		   sessions.csv
sample_submission_NDF.csv  test_users.csv


In [62]:
def detect_nan_strings(df):
    for column in df.columns:
        print("\n\ncolumn: {}:".format(column))
        print("-" * 20)
        try:
            print("is NAN string:\n", any(df[column] == 'NAN'))
            print("is NAN string:\n", any(df[column] == 'nan'))
            print("is NAN string:\n", any(df[column] == 'NaN'))
        except TypeError:
            print("column {} not str type".format(column))

In [66]:
def uniques_count(df):
    return {column: df[column].count() for column in df.columns}

In [4]:
age_gender_df = pd.read_csv('../input/age_gender_bkts.csv')

In [5]:
countries_df = pd.read_csv('../input/countries.csv')

In [6]:
sessions_df = pd.read_csv('../input/sessions.csv')

In [7]:
train_users_2_df = pd.read_csv('../input/train_users_2.csv')

In [8]:
test_users_df = pd.read_csv('../input/test_users.csv')

In [9]:
age_gender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 5 columns):
age_bucket                 420 non-null object
country_destination        420 non-null object
gender                     420 non-null object
population_in_thousands    420 non-null float64
year                       420 non-null float64
dtypes: float64(2), object(3)
memory usage: 16.5+ KB


In [10]:
age_gender_df.head()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


In [11]:
age_gender_df.describe()

,population_in_thousands,year
count,420.000000,420.0
mean,1743.133333,2015.0
std,2509.843202,0.0
min,0.000000,2015.0
25%,396.500000,2015.0
50%,1090.500000,2015.0
75%,1968.000000,2015.0
max,11601.000000,2015.0


In [12]:
countries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
country_destination              10 non-null object
lat_destination                  10 non-null float64
lng_destination                  10 non-null float64
distance_km                      10 non-null float64
destination_km2                  10 non-null float64
destination_language             10 non-null object
language_levenshtein_distance    10 non-null float64
dtypes: float64(5), object(2)
memory usage: 640.0+ bytes


In [13]:
countries_df

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00


In [14]:
sessions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


In [15]:
sessions_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [16]:
sessions_df.describe()

,secs_elapsed
count,1.043171e+07
mean,1.940581e+04
std,8.888424e+04
min,0.000000e+00
25%,2.290000e+02
50%,1.147000e+03
75%,8.444000e+03
max,1.799977e+06


In [17]:
train_users_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 26.1+ MB


In [18]:
train_users_2_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [19]:
train_users_2_df.describe()

,timestamp_first_active,age,signup_flow
count,2.134510e+05,125461.000000,213451.000000
mean,2.013085e+13,49.668335,3.267387
std,9.253717e+09,155.666612,7.637707
min,2.009032e+13,1.000000,0.000000
25%,2.012123e+13,28.000000,0.000000
50%,2.013091e+13,34.000000,0.000000
75%,2.014031e+13,43.000000,0.000000
max,2.014063e+13,2014.000000,25.000000


In [20]:
sessions_df_not_na = sessions_df.notna()

In [21]:
sessions_df.shape

(10567737, 6)

In [22]:
for column in sessions_df_not_na.columns:
    print("\n", column)
    not_na_len = len(sessions_df_not_na[sessions_df_not_na[column]])
    print("not_na:", not_na_len, " diff shape and not_na:", sessions_df.shape[0] - not_na_len)


 user_id
not_na: 10533241  diff shape and not_na: 34496

 action
not_na: 10488111  diff shape and not_na: 79626

 action_type
not_na: 9441533  diff shape and not_na: 1126204

 action_detail
not_na: 9441533  diff shape and not_na: 1126204

 device_type
not_na: 10567737  diff shape and not_na: 0

 secs_elapsed
not_na: 10431706  diff shape and not_na: 136031


In [23]:
sessions_df.shape

(10567737, 6)

In [24]:
train_users_2_df_not_na = train_users_2_df.notna()

In [25]:
for column in train_users_2_df_not_na.columns:
    print("\n", column)
    not_na_len = len(train_users_2_df_not_na[train_users_2_df_not_na[column]])
    print("not_na:", not_na_len, " diff shape and not_na:", train_users_2_df_not_na.shape[0] - not_na_len)


 id
not_na: 213451  diff shape and not_na: 0

 date_account_created
not_na: 213451  diff shape and not_na: 0

 timestamp_first_active
not_na: 213451  diff shape and not_na: 0

 date_first_booking
not_na: 88908  diff shape and not_na: 124543

 gender
not_na: 213451  diff shape and not_na: 0

 age
not_na: 125461  diff shape and not_na: 87990

 signup_method
not_na: 213451  diff shape and not_na: 0

 signup_flow
not_na: 213451  diff shape and not_na: 0

 language
not_na: 213451  diff shape and not_na: 0

 affiliate_channel
not_na: 213451  diff shape and not_na: 0

 affiliate_provider
not_na: 213451  diff shape and not_na: 0

 first_affiliate_tracked
not_na: 207386  diff shape and not_na: 6065

 signup_app
not_na: 213451  diff shape and not_na: 0

 first_device_type
not_na: 213451  diff shape and not_na: 0

 first_browser
not_na: 213451  diff shape and not_na: 0

 country_destination
not_na: 213451  diff shape and not_na: 0


In [26]:
for column in sessions_df.columns:
    print("\n", column)
    column_unique = sessions_df[column].unique()
    print("uniques", len(column_unique))
    print(column_unique)


 user_id
135484
['d1mm9tcy42' 'yo8nz8bqcq' '4grx6yxeby' ... 'fa6260ziny' '87k0fy4ugm'
 '9uqfg8txu3']

 action
360
['lookup' 'search_results' 'personalize' 'index' 'similar_listings'
 'ajax_refresh_subtotal' 'show' 'header_userpic' 'ask_question' nan
 'other_hosting_reviews_first' 'hosting_social_proof' 'decision_tree'
 'recent_reservations' 'faq_experiment_ids' 'multi' 'active' 'dashboard'
 'create' 'confirm_email' 'show_personalize' 'verify' 'pending'
 'requested' 'concierge' 'faq' 'clear_reservation' 'cancellation_policies'
 'track_page_view' 'update' 'my' 'campaigns' 'notifications' 'listings'
 'unavailabilities' 'ajax_lwlb_contact' 'ajax_check_dates' 'qt2'
 'request_new_confirm_email' 'ajax_photo_widget_form_iframe'
 'facebook_auto_login' 'identity' 'qt_reply_v2' 'travel_plans_current'
 'complete_status' 'populate_from_facebook' 'kba_update' 'kba' 'login'
 'authenticate' 'calendar_tab_inner2' 'other_hosting_reviews'
 'social_connections' 'relationship' '15' 'collections' '12'
 'ju

14
['Windows Desktop' '-unknown-' 'Mac Desktop' 'Android Phone' 'iPhone'
 'iPad Tablet' 'Android App Unknown Phone/Tablet' 'Linux Desktop' 'Tablet'
 'Chromebook' 'Blackberry' 'iPodtouch' 'Windows Phone' 'Opera Phone']

 secs_elapsed
337662
[3.190000e+02 6.775300e+04 3.010000e+02 ... 2.870570e+05 1.551558e+06
 1.752436e+06]


In [27]:
for column in train_users_2_df.columns:
    print("\n", column)
    column_unique = train_users_2_df[column].unique()
    print("uniques", len(column_unique))
    print(column_unique)


 id
213451
['gxn3p5htnn' '820tgsjxq7' '4ft3gnwmtx' ... '6o3arsjbb4' 'jh95kwisub'
 'nw9fwlyb5f']

 date_account_created
1634
['2010-06-28' '2011-05-25' '2010-09-28' ... '2014-06-27' '2014-06-29'
 '2014-06-30']

 timestamp_first_active
213451
[20090319043255 20090523174809 20090609231247 ... 20140630235754
 20140630235822 20140630235824]

 date_first_booking
1977
[nan '2010-08-02' '2012-09-08' ... '2015-06-25' '2015-06-29' '2015-06-28']

 gender
4
['-unknown-' 'MALE' 'FEMALE' 'OTHER']

 age
128
[      nan 3.800e+01 5.600e+01 4.200e+01 4.100e+01 4.600e+01 4.700e+01
 5.000e+01 3.600e+01 3.700e+01 3.300e+01 3.100e+01 2.900e+01 3.000e+01
 4.000e+01 2.600e+01 3.200e+01 3.500e+01 5.900e+01 4.900e+01 4.400e+01
 3.400e+01 2.800e+01 1.900e+01 5.300e+01 5.200e+01 3.900e+01 5.700e+01
 2.500e+01 5.400e+01 6.900e+01 6.300e+01 4.300e+01 5.500e+01 6.500e+01
 5.800e+01 6.100e+01 1.800e+01 5.000e+00 2.700e+01 4.500e+01 6.000e+01
 4.800e+01 5.100e+01 6.400e+01 7.200e+01 7.000e+01 6.700e+01 7.300e+01
 2.0

plt.figure(figsize=(16,6))
features = sessions_df.columns
plt.title("Distribution of mean values per row in the train")
sns.distplot(sessions_df[features].mode(axis=1),color="green", kde=True,bins=120, label='sessions')
#sns.distplot(test_df[features].mean(axis=1),color="blue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

features = sessions_df.columns
plt.figure(figsize=(16,6))
plt.title("Distribution of mean values per column in the sessions set")
sns.distplot(sessions_df[features].mean(axis=0),color="magenta",kde=True,bins=120, label='train')
#sns.distplot(test_df[features].mean(axis=0),color="darkblue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

In [29]:
sessions_df.shape

(10567737, 6)

In [28]:
sessions_df.isna().sum()

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

In [31]:
sessions_df.isna().sum() / len(sessions_df) * 100

user_id           0.326428
action            0.753482
action_type      10.657003
action_detail    10.657003
device_type       0.000000
secs_elapsed      1.287229
dtype: float64

In [39]:
for column in sessions_df.columns:
    print("\n\ncolumn: {}:".format(column))
    print("-" * 20)
    print("value_counts:\n", sessions_df[column].value_counts())



column: user_id:
--------------------
value_counts:
 mxqbh3ykxl    2722
0hjoc5q8nf    2644
mjbl6rrj52    2476
l5lgm3w5pc    2424
wg9413iaux    2362
ht8alhs4lt    2335
wyv1imf8qw    2323
monrpvx2md    2264
9z4gim1s4l    2264
h0cjxc177k    2246
a0uhiojrra    2137
vcmr2jh5ix    2085
1m6xnhstmb    2019
p1183hxzc4    1938
e8h4qghxlg    1923
gey51ednme    1919
5vpuk5mssg    1876
j2cvctvqve    1861
yu5bdalz2b    1811
ejpe95pcyo    1797
r541x78s24    1792
qkbkunyzq7    1780
n4s6g3grzf    1779
bfiueza7rt    1753
b1io359wpg    1752
8ikl7vnfa3    1732
e81qfos71y    1701
s5ez13snz0    1685
93dulcecw0    1614
r0rgjqbsvp    1612
              ... 
jxdfl1ivgr       1
e3vw8ung65       1
d0kmmhg1s7       1
mr2chdr4vp       1
d8ccqhesf6       1
ul7iepe9l4       1
hnp5nfnfn3       1
l1g7as8d62       1
ou812mptsh       1
sb5ji18bpl       1
8wp91p0kk0       1
qhjmr33xzy       1
6vei32cuik       1
l3khskxias       1
7nic82lz8s       1
8g392uznwi       1
imt2biwj08       1
0xep7562jh       1
7pb6pzye9v    

In [40]:
train_users_2_df.shape

(213451, 16)

In [41]:
train_users_2_df.isna().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

In [42]:
train_users_2_df.isna().sum() / len(train_users_2_df) * 100

id                          0.000000
date_account_created        0.000000
timestamp_first_active      0.000000
date_first_booking         58.347349
gender                      0.000000
age                        41.222576
signup_method               0.000000
signup_flow                 0.000000
language                    0.000000
affiliate_channel           0.000000
affiliate_provider          0.000000
first_affiliate_tracked     2.841402
signup_app                  0.000000
first_device_type           0.000000
first_browser               0.000000
country_destination         0.000000
dtype: float64

In [43]:
for column in train_users_2_df.columns:
    print("\n\ncolumn: {}:".format(column))
    print("-" * 20)
    print("value_counts:\n", train_users_2_df[column].value_counts())



column: id:
--------------------
value_counts:
 84v31mmg92    1
00allnceb8    1
oeqyzp9fml    1
6pgtq5qm3a    1
hqenwsqojy    1
prqnudhgdx    1
ujbug5f7v6    1
fgthxap8kb    1
v9nborqc1s    1
8nxk7ratyc    1
ync50l8x3i    1
j1j8dy8out    1
63kvn42tw7    1
8roixk9uo5    1
ote13e7teg    1
9843v3nvp6    1
rmh7xzvfl7    1
t8i1f4k63a    1
cwfnbfvr6f    1
1xle71ems8    1
h1av7x305n    1
ae7xxaf5mf    1
fzgeceo0i3    1
45narjgdw1    1
h6gmjr53vh    1
5te72hp164    1
ahv7137now    1
s2mbwokl06    1
yseei1191j    1
4h96mvj6ts    1
             ..
0yguc7k3um    1
3c4dja5tlz    1
2pqg1kot4e    1
eaao7fkr3i    1
ub3wjs33ts    1
6ib45u4wl9    1
fb06fvswc6    1
xx0tqida65    1
afgqftmmwb    1
grn1bxkzgs    1
qoeseq5eeq    1
7lp2beb628    1
vs4xb89rep    1
cjv70yb1l5    1
28j4etyzsh    1
lc87e7g2rq    1
jg8byy20da    1
ojbt7varol    1
292l7q8ako    1
4hq0083zi1    1
rzlqc3wq6o    1
rht3khyr7k    1
kx3gb5qldt    1
e4geou2ml2    1
07watt77xk    1
appkm91d2b    1
cmlud0s9xr    1
2nswkg2py1    1
485ofy

In [45]:
for column in sessions_df.columns:
    print("\n\ncolumn: {}:".format(column))
    print("-" * 20)
    try:
        print("is nan string:\n", any(sessions_df[column] == 'nan'))
    except TypeError:
        print("column {} not str type".format(column))



column: user_id:
--------------------
is nan string:
 False


column: action:
--------------------
is nan string:
 False


column: action_type:
--------------------
is nan string:
 False


column: action_detail:
--------------------
is nan string:
 False


column: device_type:
--------------------
is nan string:
 False


column: secs_elapsed:
--------------------
column secs_elapsed not str type


In [46]:
for column in sessions_df.columns:
    print("\n\ncolumn: {}:".format(column))
    print("-" * 20)
    try:
        print("is NAN string:\n", any(sessions_df[column] == 'NAN'))
    except TypeError:
        print("column {} not str type".format(column))



column: user_id:
--------------------
is NAN string:
 False


column: action:
--------------------
is NAN string:
 False


column: action_type:
--------------------
is NAN string:
 False


column: action_detail:
--------------------
is NAN string:
 False


column: device_type:
--------------------
is NAN string:
 False


column: secs_elapsed:
--------------------
column secs_elapsed not str type


In [48]:
for column in sessions_df.columns:
    print("\n\ncolumn: {}:".format(column))
    print("-" * 20)
    try:
        print("is NaN string:\n", any(sessions_df[column] == 'NaN'))
    except TypeError:
        print("column {} not str type".format(column))



column: user_id:
--------------------
is NaN string:
 False


column: action:
--------------------
is NaN string:
 False


column: action_type:
--------------------
is NaN string:
 False


column: action_detail:
--------------------
is NaN string:
 False


column: device_type:
--------------------
is NaN string:
 False


column: secs_elapsed:
--------------------
column secs_elapsed not str type


In [50]:
sessions_df_unique_user_ids = set(sessions_df['user_id'].unique())

In [51]:
train_users_2_df_unique_user_ids = set(train_users_2_df['id'].unique())

In [52]:
len(sessions_df_unique_user_ids)

135484

In [53]:
len(train_users_2_df_unique_user_ids)

213451

In [54]:
len(sessions_df_unique_user_ids.union(train_users_2_df_unique_user_ids))

275120

In [55]:
len(sessions_df_unique_user_ids.difference(train_users_2_df_unique_user_ids))

61669

In [56]:
len(train_users_2_df_unique_user_ids.difference(sessions_df_unique_user_ids))

139636

In [57]:
len(train_users_2_df_unique_user_ids.intersection(sessions_df_unique_user_ids))

73815

Preliminary results:
1. We can see that some columns in sessions data and users data have signficant numbers of NaN values.
sessions:
user_id           0.326428% (which is curious in itself)
action            0.753482%
action_type      10.657003%
action_detail    10.657003%
device_type       0.000000%
secs_elapsed      1.287229%

users:
id                          0.000000%
date_account_created        0.000000%
timestamp_first_active      0.000000%
date_first_booking         58.347349%
gender                      0.000000%
age                        41.222576%
signup_method               0.000000%
signup_flow                 0.000000%
language                    0.000000%
affiliate_channel           0.000000%
affiliate_provider          0.000000%
first_affiliate_tracked     2.841402%
signup_app                  0.000000%
first_device_type           0.000000%
first_browser               0.000000%
country_destination         0.000000%

2. user_id column in sessions data contain 0.326428% NaNs, rows with will be deleted when we  will simultaneously use data from sessions and users datasets (or may be we find the way fill this values).
3. Deciding what to do with the NaNs in columns action_type and action_detail (Delete or fill by average, mode or by values that we can retrieve using simple ML models) requires additional exploring, but now we delete this rows
4. NaNs in secs_elapsed in session data we can fill by mean or mode.
5. NaNs in first_affiliate_tracked in users data make sense delete or fill by mode.
6. NaNs in date_first_booking requires additional eploring. Or delete whole column if additional exploring don't give result.
7. Same for age column.
8. It is also worth paying attention to the fact that the set of user IDs in sessions dataset and users dataset  do not match, but only intersect in some part of thes. There are users whose IDs are in session dataset  but not in users dataset, which
strange.
9. It is also worthwhile to look in more detail at the distribution of the number of unique values in each column of each dataset.
10. Add columns with timestamps converted in datetime format.

In [58]:
sessions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


In [59]:
train_users_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 26.1+ MB


In [60]:
train_users_2_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [63]:
sessions_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [64]:
sessions_df.count()

user_id          10533241
action           10488111
action_type       9441533
action_detail     9441533
device_type      10567737
secs_elapsed     10431706
dtype: int64